<p style="font-size: 24pt; font-weight: 500;"}>The Grand Tour for Outlier Detection</p>
<hr>
\author{Stanley Speel}

This document aims to demonstrate that *The Grand Tour* can be used as an effective and elegant multidimensional outlier detection technique.  Though it does not offer any accuracy improvements over other detection methods, it provides a uniquely transparent and interpretable view of the detection process.

# The Grand Tour Method

## High Level Overview 

*The Grand Tour* (TGT) was a popular technique for visualising high dimensional data, first introduced by Daniel Asimov \cite{TGT}. It involves continually projecting a data set into two-dimensional subspaces in such a way that, when considered over time, provides a continuous trajectory of the subspace. The method is covered mathematically in Section 1.2. 

For the purposes of introducing the method, it is useful to first consider a low-dimensional data set that can be visualised. Consider thus, an arbitrary data set with three input columns (variables). Such data may be visualised by means of a three-dimensional scatterplot, with each perpendicular axis representing one column. The figure below depicts a trivial data set with two data points. By considering the *projection* of the data along two different axes, it is apparent that any such two-dimensional projection will likely lose much of the information contained within the three-dimensional space. 

![3D_Axes](images/3D_Axes.png "3D Axes")

A projection need not be aligned with any particular axes; any point in the three-dimensional space can act as an observation point, generating a unique two-dimensional projection. This is much the same as a camera capturing a two-dimensional view of the three-dimensional world it lives in - the camera can be anywhere in space, and the image generated will appear different depending on its orientation. The number of distinctly different projections - projections that differ by more than just scale - is defined as the set of points defined by a sphere with a constant radius, centred at our data.

Although there are infinitely many projections of the two three-dimensional points, it does not take many to garner a sense of the three-dimensional data under consideration - a human or stereoscopic camera can reconstruct three-dimensional data from just two views \cite{Hartley}. It is thus possible to conclude that, by analysing a very small subset of available two-dimensional projections, it is possible to extract the information contained within the full three-dimensional space. 

This concept may be extended to much higher dimensions, albeit with a few further considerations. For an $N$-dimensional data set (i.e. a data set with $N$ columns), the number of distinctly different two-dimensional projections is much higher - much more information is lost in every projection. Nonetheless, it is not unreasonable to assume that higher dimensions will behave similarly to three-dimensional space; i.e. one need only consider a small subset of projections to reconstruct the full data distribution. 

The problem of subset selection then arises - with so much information being lost with each projection, how can one be sure that the selected projections have fairly sampled the high-dimensional data? Asimov showed that by incrementally moving the projection, and analysing the sequence of these projections over continuous time, the information that can be gained from this analyis will converge to the information that can be gained by the analysis of all possible two-dimensional projections of the space \cite{TGT}. For example, in the figure below, by investigating the relative velocities of the two-dimensional point projections it is possible to determine their relative distances. 
<br>
<img src="images/moving_points.gif" width="250">

The Grand Tour method utilises this phenomenon, providing an apparently continuous two-dimensional projection sequence of high-dimensional data. A caveat of high dimensions is that (mathematical) care must be taken to provide a continuous movement of the two-dimensional projection. 

## Mathematical Explanation

Consideration of a mathematical overview of the method formalises the concepts introduced above, and addresses the continuity issues that are faced in high dimensions. 

For a data set with N columns, the set $S$ of all possible structurally unique two-dimensional projections lie on an N-dimensional hypersphere of unit radius, with its centre $O$ lying at the average of all the data points. A two-dimensional plane $\Pi_{OUP}$ is defined that intersects this centre, as well as two randomly chosen points on the hypersphere, $U$ and $P$. A rotation matrix $R$ is defined that will rotate the data set around the hypersphere, confined to the surface of the two-dimensional plane. 

<img src="images/hypersphere.gif" width="700">

Rotating the data around the plane $\Pi_{OUP}$ ensures that during a given rotation, a continuous and smooth transition of the data points' projection onto any two axes is observed. Once the data set has rotated the angle spanned by $U$ and $P$, a new point $U_{n}$ and respective plane $\Pi_{OU_{n}P}$ is defined. 

The below animation shows a 7 dimensional data \cite{Glass} set being subjected to such rotations and projections. Severe outliers quickly become apparent to the human observer, and thus it is plausible to consider their detection to be an automated process. 

<img src="images/point_cloud.gif" width="400">

The generation process of the rotation matrix $R$ such that the rotation is aligned with the plane $\Pi_{OUP}$ is not entirely straighforward, and unfortunately there seemed to be no existing implementation. An algebraic sketch proof and Python implementation of the generation of $R$ is thus included in the appendix of this document.

# Utilising TGT Projections to Detect Outliers

## Existing research

In contrast to the extensive coverage of The Grand Tour as a visualisation tecnique, research exploring its use for the detection of outliers is much more scarce. There has been some research which fitted *concentration ellipses* - for all intents and purposes a two-dimensional Gaussian - over the projections \cite{TGTOutlier}. The *time* in which each point spent outside of the ellipses was then recorded for each frame.

The figure below aims to clarify this concept. Cosnider the depicted sequence of projections, generated using The Grand Tour method. The greater the 'degree' of an outlier, the longer its expected trajctory outside a fitted ellipse. Thus, if the sequence is allowed to run for a sufficiently long time, the time each point spends outside of the fitted ellipse may be conidered a measure of 'outlierness'.

<img src="images/outlier_trajectories.png" width="1000">

The conclusions of \cite{TGTOutlier} were that the method works well for data that is approximately elliptically distributed, and that this is indeed the case for many data sets. With the research being over 20 years old, a significant bottleneck was computational cost - there was no attempt to fit any other distributions to the projections.

## Research Goals

With a significant increase in computational power, there is no reason to be restricted to univariate Gaussian fits to the two-dimensional projections. A number of different fitting functions are therefore tested over 11 benchmark outlier data sets \cite{Campos2016}. 

The overall research goals were to find a fitting function that is flexible enought to generalise well to different data sets, whilst also being constrained enough so as not to overfit to outliers that would be obvious to the human observer.

## Gaussian Mixture Fitting Functions 

A two and three component Gaussian Mixture Model (GMM) with shrinkage seemed to perform best when averaged over the benchmark data sets. The figure below depicts outliers being detected using a two-component GMM on a seven dimensional data set. Outliers are depicted as large points, with the right-hand plot showing only the outliers.

<img src="images/GMM.gif" width="500">

As may be expected, the number of outliers detected depends on the standard-deviation boundary of the fitted Gaussians. One thing that was noted, however, was that the convergence rate of the detected outliers was fast on all data sets trialled (a few iterations) regardless of the standard deviation boundry selected. Relative performances are discussed in Section 3.

## Sequential 2D Outlier Detection Methods

Other fitting functions trialled included GMMs with a greater degree of freedom, as well as a number of existing outlier detection algorithms (on the two-dimensional projections). These included Histogram-Based Outlier Detection, Feature Bagging, Local Outlier Factor, and Local Outlier Probability. 

Local Outlier Probability (LoOP) performed respectably due to its stringency in predicting outliers, however the detection process was performed independently on the two axes of the projections. Unfortunately, high sensitivity to the probabilistic boundaries used to detect whether or not a point in a projection is an outlier rendered this method unstable, and thus was not investigated further.

# Results

There is some inherent subjectiveness in determining whether a point in a data set is an outlier. Therefore, obtaining a robust ground truth for quantitative assessment of outlier detection methods is difficult; there is no 'gold standard' outlier detection algorithm. 

\textit{Area under the ROC curve} (AUC) can be used as a performance metric for outlier detection methods \cite{Campos2016} when a ground truth exists, however it is perhaps more an indication of algorithmic \textit{potential} as opposed to raw performance. It assumes that, for each data set, an optimal probabilistic boundary can be determined to classify points as outliers or otherwise.

The below figure shows the AUC scores of different outlier detection metrics on three benchmark data sets, referenced from  \cite{PLOS}. TGT outlier detection scores are included using a two-component Gaussian Mixture Model, and Gaussian.

<img src="images/TGT_Results.png" width="1000">

It can be seen that performance is heavily dataset dependent - in the WDBC and WBC data sets it performed comparably to other popular methods, whereas in the Glass data set it performed considerably worse ( all data sets from \cite{Campos2016} ).

# Discussion and Conclusions

Using The Grand Tour for outlier detection rapidly highlights 'obvious' outliers in a multidimensional data set in a visually intuitive and transparent way. An attempt to quantify its performance against other multidimensional outlier detection methods using AUC scores shows that, when compared to a set of benchmark data sets, there are no significant imrovements in detection accuracy. 

It should however be noted that different outlier detection techniques inherently highlight different types of outliers (see LoOP SRP). The figure below was taken from \cite{PLOS}, a comprehensive study of the relative merits of other multidimensional outlier detection methods. The paper concludes that there is no 'one-size-fits all' outlier detection method, thus the selection of an algorithm within AuDaS should be specifically tailored to the desired type of outlier to be detected, and the format of any output to the user.

<img src="images/algorithm_selection.png" width="900">


# References

[<a id="cit-TGT" href="#call-TGT">1</a>] Asimov Daniel, ``_The Grand Tour: A Tool for Viewing Multidimensional Data_'', Siam Journal on Scientific and Statistical Computing, vol. 6, number , pp. , 01 1985.

[<a id="cit-Hartley" href="#call-Hartley">2</a>] Richard Hartley and Andrew Zisserman, ``_Multiple View Geometry in Computer Vision_'',  2000.

[<a id="cit-Glass" href="#call-Glass">3</a>] F. Keller, E. Muller and K. Bohm, ``_HiCS: High Contrast Subspaces for Density-Based Outlier Ranking_'', 2012 IEEE 28th International Conference on Data Engineering, April 2012.

[<a id="cit-TGTOutlier" href="#call-TGTOutlier">4</a>] Bartkowiak Anna and Szustalewicz Adam, ``_The Grand Tour as a Method for Detecting Multivariate Outliers_'', , vol. , number , pp. , 09 1998.

[<a id="cit-Campos2016" href="#call-Campos2016">5</a>]  , ``_On the evaluation of unsupervised outlier detection: measures, datasets, and an empirical study_'', Data Mining and Knowledge Discovery, vol. 30, number 4, pp. 891--927, Jul 2016.  [online](https://doi.org/10.1007/s10618-015-0444-8)

